In [1]:
import pandas as pd
import numpy as np
import glob
import os
import cv2
from tensorflow.keras.models import model_from_json


In [2]:
current_dir = "/INGVDATA/WFQC/"
model_dir = current_dir + "Model/"
data_dir = current_dir + "Data/"
spectrogram_dir = data_dir + "Extracted_Spectrogram_Full_Analysis/" 
output_dir = current_dir + "Output/"

In [3]:
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

In [4]:
spectrogram_filenames = glob.glob(spectrogram_dir + '/*.png')

In [5]:
with open(model_dir + 'cnn_architecture_all_data.json', 'r') as f:
    model_cnn = model_from_json(f.read())
model_cnn.load_weights(model_dir + 'cnn_weights_all_data.h5')

with open(model_dir + 'vgg16_architecture_all_data.json', 'r') as f:
    model_vgg16 = model_from_json(f.read())
model_vgg16.load_weights(model_dir + 'vgg16_weights_all_data.h5')

with open(model_dir + 'ResNet50_architecture_all_data.json', 'r') as f:
    model_ResNet50 = model_from_json(f.read())
model_ResNet50.load_weights(model_dir + 'ResNet50_weights_all_data.h5')

with open(model_dir + 'DenseNet121_architecture_all_data.json', 'r') as f:
    model_DenseNet121 = model_from_json(f.read())
model_DenseNet121.load_weights(model_dir + 'DenseNet121_weights_all_data.h5')



2021-11-09 22:04:25.804109: E tensorflow/stream_executor/cuda/cuda_driver.cc:271] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
2021-11-09 22:04:25.804188: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (geoaidmax): /proc/driver/nvidia/version does not exist
2021-11-09 22:04:25.804981: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [6]:
ncol, nrow = 300, 300

full_analysis_score = pd.DataFrame()
full_analysis_score['spectrogram_filename'] = spectrogram_filenames
full_analysis_score['waveform_filename'] = ''
full_analysis_score['spectrogram_start_second'] = ''
full_analysis_score['predicted_probability'] = 0.0


In [7]:
opt_weights = pd.read_excel(output_dir + 'opt_weights.xlsx', header = None, engine='openpyxl')[0].values.tolist()


In [8]:
print(opt_weights)


[0.2535064697265625, 0.2530548095703125, 0.233355712890625, 0.2600830078125]


In [ ]:


res_data = []

for index, row in full_analysis_score.iterrows():
    if (index % 10000 == 0):
        print([index,row])
    simplefile=os.path.basename(row['spectrogram_filename'])
    sta,chan,sub,sti=simplefile.split('_')
    stim,ext=sti.split('.')
    stim=stim[1:]
    #print(stim)
    waveform_file=sub+'/'+sta+'.'+chan+'.vel'
    #print(waveform_file)
    img = cv2.imread(row['spectrogram_filename'])
    img = cv2.resize(img, (ncol, nrow))
    img_reshaped = []
    img_reshaped.append(img)
    predict_prob_cnn = model_cnn.predict(np.asarray(img_reshaped) / 255.0).tolist()[0][0]
    predict_prob_vgg16 = model_vgg16.predict(np.asarray(img_reshaped) / 255.0).tolist()[0][0]
    predict_prob_ResNet50 = model_ResNet50.predict(np.asarray(img_reshaped) / 255.0).tolist()[0][0]
    predict_prob_DenseNet121 = model_DenseNet121.predict(np.asarray(img_reshaped) / 255.0).tolist()[0][0]
    predicted_probability = sum([x*y for x,y in zip([predict_prob_cnn, predict_prob_vgg16, predict_prob_ResNet50, predict_prob_DenseNet121], opt_weights)]) 
        ## the opmized weight for each model was computed in previous step
    res_data.append([simplefile,waveform_file,stim,predicted_probability])


[0, spectrogram_filename        /INGVDATA/WFQC/Data/Extracted_Spectrogram_Full...
waveform_filename                                                            
spectrogram_start_second                                                     
predicted_probability                                                     0.0
Name: 0, dtype: object]


2021-11-09 22:05:03.148093: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enabled (registered 2)
